In [4]:
import pandas as pd
import openai
import json
import time
import matplotlib.pyplot as plt
from tqdm import tqdm  # Import tqdm for the progress bar
from sklearn.metrics import accuracy_score, classification_report, balanced_accuracy_score
from sklearn.metrics import balanced_accuracy_score
import numpy as np

In [5]:
#pip install openai

In [6]:
from openai import OpenAI

In [7]:
openai.__version__


'1.14.0'

### Read in annotated data

In [19]:
df = pd.read_csv("../data/data_splits_stratified/6-2-2_all_classes_enriched_with_kw/test.csv", index_col=0)
#df = pd.read_csv("../data/data_splits_stratified/6-2-2_all_classes_enriched/test.csv", index_col=0)

In [23]:
df.shape

(534, 10)

In [20]:
df.head()

,pmid,journal_name,title,abstract,keywords,accepted_label,multi_label,binary_label
idx,,,,,,,,
1,12047012,Schizophrenia bulletin,Can clinical practice guide a research agenda?,Articles from this issue of the Bulletin indic...,NaN,Non-systematic-review,1,0
2,28832188,Future medicinal chemistry,Dual/multitargeted xanthone derivatives for Al...,"To date, the current therapy for Alzheimer's d...",Alzheimer's disease| Aβ aggregation| anticholi...,Non-systematic-review,1,0
3,17678496,Expert review of neurotherapeutics,Benefits of occupational therapy in stroke reh...,Stroke is the largest single cause of severe p...,NaN,Non-systematic-review,1,0
4,25649308,Annals of the New York Academy of Sciences,The promise of ketamine for treatment-resistan...,Major depressive disorder (MDD) is one of the ...,antidepressant| bipolar disorder| ketamine| ma...,Non-systematic-review,1,0
5,6312596,La semaine des hopitaux : organe fonde par l'A...,[Clinical and pathogenic aspects of secondary ...,The secondary hyperlipoproteinemias are freque...,NaN,Non-systematic-review,1,0


In [24]:
# Combine the columns
# Implementing custom tags for the combination of journal name, title, and abstract
df['input_journal_title_abstract'] = '<journal>' + df['journal_name'] + '</journal>' + \
                                     '<title>' + df['title'] + '</title>' + \
                                     '<abstract>' + df['abstract'] + '</abstract>'

# Implementing custom tags for the combination of title and abstract only
df['input_title_abstract'] = '<title>' + df['title'] + '</title>' + \
                             '<abstract>' + df['abstract'] + '</abstract>'

In [25]:
df.head(5)

,pmid,journal_name,title,abstract,keywords,accepted_label,multi_label,binary_label,input_journal_title_abstract,input_title_abstract
idx,,,,,,,,,,
1,12047012,Schizophrenia bulletin,Can clinical practice guide a research agenda?,Articles from this issue of the Bulletin indic...,NaN,Non-systematic-review,1,0,<journal>Schizophrenia bulletin</journal><titl...,<title>Can clinical practice guide a research ...
2,28832188,Future medicinal chemistry,Dual/multitargeted xanthone derivatives for Al...,"To date, the current therapy for Alzheimer's d...",Alzheimer's disease| Aβ aggregation| anticholi...,Non-systematic-review,1,0,<journal>Future medicinal chemistry</journal><...,<title>Dual/multitargeted xanthone derivatives...
3,17678496,Expert review of neurotherapeutics,Benefits of occupational therapy in stroke reh...,Stroke is the largest single cause of severe p...,NaN,Non-systematic-review,1,0,<journal>Expert review of neurotherapeutics</j...,<title>Benefits of occupational therapy in str...
4,25649308,Annals of the New York Academy of Sciences,The promise of ketamine for treatment-resistan...,Major depressive disorder (MDD) is one of the ...,antidepressant| bipolar disorder| ketamine| ma...,Non-systematic-review,1,0,<journal>Annals of the New York Academy of Sci...,<title>The promise of ketamine for treatment-r...
5,6312596,La semaine des hopitaux : organe fonde par l'A...,[Clinical and pathogenic aspects of secondary ...,The secondary hyperlipoproteinemias are freque...,NaN,Non-systematic-review,1,0,<journal>La semaine des hopitaux : organe fond...,<title>[Clinical and pathogenic aspects of sec...


### Load key for the OpenAI API 

In [26]:
def load_pass(file_path, key_to_find):
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split("=")
            if len(parts) == 2 and parts[0] == key_to_find:
                found_password = parts[1]
                break
    if found_password:
        print("Found password.")
        return found_password
    else:
        print("Password not found for key:", key_to_find)

Note: You need to create a credentials.txt file with the following content:  
OPENAI=sk-77QXXXXXXXXXXXXXXXXXXXXXXXXXXX  
replace the value after the = sign with your API key.  
Make sure the credentials.txt is added to .gitignore, you don't want to put your password on Git!

In [27]:
openai.api_key = load_pass("./credentials.txt", "OPENAI")


Found password.


In [28]:
client = OpenAI(api_key=openai.api_key)

### Query GPT

To change the task the model is solving, you need to change the text of the prompt and the content text of the system role.  
To change the GPT model used, you need to change the text of the model name when initiating the openai API.  
The function gets as input the input_raw_text, that will be text for information extraction or classification.

In [29]:
import time

DEFAULT_TEMPERATURE = 0
DEFAULT_MAX_TOKENS = 500
DEFAULT_MODEL = "gpt-3.5-turbo"

def query_gpt(input_raw_text, prompt_text, gpt_model="gpt-3.5-turbo", temperature=0, max_retries=5, retry_delay=3):
    # CHANGE gpt_model to the desired model name, see https://platform.openai.com/docs/models (gpt-3.5-turbo and gpt-4-turbo-preview)
    
    # Add a delay at the beginning of the function to avoid overloading the API if there are multiple calls
    # time.sleep(10)  

    system_msg = f"""
    You are an expert assistant specialized in text classification of PubMed abstracts. """

    retries = 0
    while retries < max_retries:
        print("Trying to call OpenAI API...")
        try:
            completion = client.chat.completions.create(
                model=gpt_model,  
                response_format={"type": "json_object"},
                temperature=temperature,
                #max_tokens=2000,
                messages=[
                    {"role": "system", "content": system_msg},
                    {"role": "user", "content": prompt_text + input_raw_text}
                ]
            )
            return completion.choices[0].message.content
        except Exception as e:
            # Handle API error, e.g., retry or log
            print(f"OpenAI API returned an error: {e}")
            time.sleep(retry_delay)  # Wait before retrying
            retries += 1

    raise RuntimeError("Max retries reached. Unable to complete the API call.")


In [30]:
# Define a function to apply GPT queries with a progress bar
def apply_gpt_with_progress(data_series, prompt_text, model="gpt-3.5-turbo"):
    results = []
    total_items = len(data_series)
    # Create a tqdm progress bar
    with tqdm(total=total_items, desc=f"Processing dataset") as pbar:
        for text in data_series:
            result = query_gpt(text, prompt_text, model)
            results.append(result)
            pbar.update(1)  # Update the progress bar

    return results

# Multi-label: Read prompts from file and query GPT

In [31]:
json_file_path = "./prompt_strategies_shirin.json"
# Load the JSON file
with open(json_file_path, 'r') as file:
    prompts_data = json.load(file)

In [ ]:
df.head() #show first few lines of df
#print(df) #shows whole df

In [32]:
## Sample to test and demo
#sampled_df = df.sample(n=50, random_state=1)
#sampled_df.head()

,pmid,journal_name,title,abstract,keywords,accepted_label,multi_label,binary_label,input_journal_title_abstract,input_title_abstract
idx,,,,,,,,,,
530,24507711,Journal of clinical neuroscience : official jo...,Validation of the Chinese version of the NUCOG...,We aimed to develop and validate a Chinese ver...,Cognition| Dementia| Epilepsy| Mini-Mental Sta...,Human-RCT-non-intervention,13,0,<journal>Journal of clinical neuroscience : of...,<title>Validation of the Chinese version of th...
404,28684367,World neurosurgery,Endovascular Therapy for the Steal Phenomenon ...,Symptomatic innominate artery stenosis present...,CAS| Carotid artery stenosis| Innominate arter...,Human-case-report,4,0,<journal>World neurosurgery</journal><title>En...,<title>Endovascular Therapy for the Steal Phen...
465,2840539,Life sciences,Elevation of naloxone-sensitive 3H-dihydromorp...,3H-Dihydromorphine (DHM) binding sites were me...,NaN,Animal-other,5,1,<journal>Life sciences</journal><title>Elevati...,<title>Elevation of naloxone-sensitive 3H-dihy...
143,35332187,Scientific reports,Effects of Apolipoprotein E polymorphism on ca...,The Apolipoprotein E (APOE) gene polymorphism ...,NaN,Remaining,0,0,<journal>Scientific reports</journal><title>Ef...,<title>Effects of Apolipoprotein E polymorphis...
522,26582408,JMIR research protocols,Western Australian Public Opinions of a Minimu...,BACKGROUND: Excessive alcohol consumption has ...,addiction| alcohol| minimum pricing| policy,Clinical-study-protocol,12,0,<journal>JMIR research protocols</journal><tit...,<title>Western Australian Public Opinions of a...


## Run different prompts over the data

In [33]:
# Add the IDs of Prompts that you want to test
prompt_ids_to_test = ["P1", "P2", "P3", "P4"] #, "P2", "P3", "P4"
model = "gpt-3.5-turbo"

for prompt in prompts_data["prompts"]:
    prompt_id = prompt["id"]
    prompt_text = prompt["text"]

    if prompt_id in prompt_ids_to_test:
        # Apply GPT predictions
        df[f'gpt_predictions_{prompt_id}_raw'] = apply_gpt_with_progress(df['input_journal_title_abstract'], prompt_text, model)
        ## the below includes error handling in case the json formatting did not work as expected
        df[f'gpt_predictions_{prompt_id}'] = df[f'gpt_predictions_{prompt_id}_raw'].apply(
            lambda x: json.loads(x)['gpt_label'] if isinstance(x, str) and 'gpt_label' in json.loads(x) else x
        )
        df.to_csv(f"predictions/{model}_outputs_{'_'.join(prompt_ids_to_test)}.csv") # saving after each prompt strategy has been run, in order not to loose information in case a later strategy fails
    else:
        print(f"Skipping prompt {prompt_id}")
        

Processing dataset:   0%|                                                                                                                                                              | 0/50 [00:00<?, ?it/s]

Trying to call OpenAI API...


Processing dataset:   2%|███                                                                                                                                                   | 1/50 [00:01<01:13,  1.50s/it]

Trying to call OpenAI API...


Processing dataset:   4%|██████                                                                                                                                                | 2/50 [00:02<00:49,  1.04s/it]

...

Trying to call OpenAI API...


Processing dataset:  98%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████   | 49/50 [00:43<00:01,  1.08s/it]

Trying to call OpenAI API...


Processing dataset: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:44<00:00,  1.14it/s]

Skipping prompt P5


In [34]:
df.head()

,pmid,journal_name,title,abstract,keywords,accepted_label,multi_label,binary_label,input_journal_title_abstract,input_title_abstract,gpt_predictions_P1_raw,gpt_predictions_P1,gpt_predictions_P2_raw,gpt_predictions_P2,gpt_predictions_P3_raw,gpt_predictions_P3,gpt_predictions_P4_raw,gpt_predictions_P4
idx,,,,,,,,,,,,,,,,,,
530,24507711,Journal of clinical neuroscience : official jo...,Validation of the Chinese version of the NUCOG...,We aimed to develop and validate a Chinese ver...,Cognition| Dementia| Epilepsy| Mini-Mental Sta...,Human-RCT-non-intervention,13,0,<journal>Journal of clinical neuroscience : of...,<title>Validation of the Chinese version of th...,"{\n ""gpt_label"": ""Human-non-RCT-non-drug-in...",Human-non-RCT-non-drug-intervention,"{\n ""gpt_label"": ""Human-non-RCT-non-drug-in...",Human-non-RCT-non-drug-intervention,"{\n ""gpt_label"": ""Clinical-study-protocol"",...",Clinical-study-protocol,"{\n ""gpt_label"": ""Human, non-RCT non-drug-i...","Human, non-RCT non-drug-intervention"
404,28684367,World neurosurgery,Endovascular Therapy for the Steal Phenomenon ...,Symptomatic innominate artery stenosis present...,CAS| Carotid artery stenosis| Innominate arter...,Human-case-report,4,0,<journal>World neurosurgery</journal><title>En...,<title>Endovascular Therapy for the Steal Phen...,"{\n ""gpt_label"": ""Human-case-report""\n}",Human-case-report,"{\n ""gpt_label"": ""Human-case-report""\n}",Human-case-report,"{\n ""gpt_label"": ""Human-case-report"",\n ...",Human-case-report,"{\n ""gpt_label"": ""Case report""\n}",Case report
465,2840539,Life sciences,Elevation of naloxone-sensitive 3H-dihydromorp...,3H-Dihydromorphine (DHM) binding sites were me...,NaN,Animal-other,5,1,<journal>Life sciences</journal><title>Elevati...,<title>Elevation of naloxone-sensitive 3H-dihy...,"{\n ""gpt_label"": ""Animal-drug-intervention""\n}",Animal-drug-intervention,"{\n ""gpt_label"": ""Animal-drug-intervention""\n}",Animal-drug-intervention,"{\n ""gpt_label"": ""Animal-other"",\n ""gpt_...",Animal-other,"{\n ""gpt_label"": ""Animal, other""\n}","Animal, other"
143,35332187,Scientific reports,Effects of Apolipoprotein E polymorphism on ca...,The Apolipoprotein E (APOE) gene polymorphism ...,NaN,Remaining,0,0,<journal>Scientific reports</journal><title>Ef...,<title>Effects of Apolipoprotein E polymorphis...,"{\n ""gpt_label"": ""Human-systematic-review""\n}",Human-systematic-review,"{\n ""gpt_label"": ""Human-systematic-review""\n}",Human-systematic-review,"{\n ""gpt_label"": ""Human-systematic-review"",...",Human-systematic-review,"{\n ""gpt_label"": ""Human, systematic review""\n}","Human, systematic review"
522,26582408,JMIR research protocols,Western Australian Public Opinions of a Minimu...,BACKGROUND: Excessive alcohol consumption has ...,addiction| alcohol| minimum pricing| policy,Clinical-study-protocol,12,0,<journal>JMIR research protocols</journal><tit...,<title>Western Australian Public Opinions of a...,"{\n ""gpt_label"": ""Clinical-study-protocol""\n}",Clinical-study-protocol,"{\n ""gpt_label"": ""Clinical-study-protocol""\n}",Clinical-study-protocol,"{\n ""gpt_label"": ""Clinical-study-protocol"",...",Clinical-study-protocol,"{\n ""gpt_label"": ""Study protocol""\n}",Study protocol


# Binary: Read prompts from file and query GPT

In [1]:
# copy the original dataframe
df_for_binary = df.copy()

# sample to to test and as demo
#sampled_df = df_for_binary.sample(n=50, random_state=1)
#sampled_df.head()

df_for_binary.head() #shows first few lines of df_for_binary

NameError: name 'df' is not defined

In [50]:
json_file_path = "./prompt_strategies_binary_shirin.json"
# Load the JSON file

with open(json_file_path, 'r') as file:
    prompts_data = json.load(file)

In [51]:
prompts_data

{'prompts': [{'id': 'P1',
   'text': "Classify this text, choosing one of these labels: 'ANIMAL' if the text is related to animal, and 'OTHER' for any other study type. Respond in json format with the key: gpt_label.",
   'strategy_type': 'zero_shot'},
  {'id': 'P2',
   'text': "Please classify the provided text by selecting the appropriate label. Use the label 'ANIMAL' if the text discusses anything related to animals, and 'OTHER' for all other types of content. Provide your response in JSON format using the key gpt_label. For instance, if the text is about animals, your response should be {gpt_label: ANIMAL}.",
   'strategy_type': 'zero_shot'},
  {'id': 'P3',
   'text': "Please classify the provided text by selecting the appropriate label. Use the label 'ANIMAL' if the text discusses anything related to animal research, such as summary of animal studies, studies testing interventions on animals, and any other studies involving animals. Use 'OTHER' for all other research types, such a

In [52]:
target_file_name_for_predictions = f"predictions/{model}_outputs_{'_'.join(prompt_ids_to_test)}_binary.csv"

### Run different prompts over the data

In [53]:
# Add the IDs of Prompts that you want to test
prompt_ids_to_test = ["P1", "P2", "P3"] #, "P2", "P3", "P4"
model = "gpt-3.5-turbo"

for prompt in prompts_data["prompts"]:
    prompt_id = prompt["id"]
    prompt_text = prompt["text"]

    if prompt_id in prompt_ids_to_test:
        # Apply GPT predictions
        df_for_binary[f'gpt_predictions_{prompt_id}_raw'] = apply_gpt_with_progress(df_for_binary['input_journal_title_abstract'], prompt_text, model)
        ## the below includes error handling in case the json formatting did not work as expected
        df_for_binary[f'gpt_predictions_{prompt_id}'] = df_for_binary[f'gpt_predictions_{prompt_id}_raw'].apply(
            lambda x: json.loads(x)['gpt_label'] if isinstance(x, str) and 'gpt_label' in json.loads(x) else x
        )
        df_for_binary.to_csv(target_file_name_for_predictions) # saving after each prompt strategy has been run, in order not to loose information in case a later strategy fails
    else:
        print(f"Skipping prompt {prompt_id}")

Processing dataset:   0%|                                                                                                                                                              | 0/50 [00:00<?, ?it/s]

Trying to call OpenAI API...


Processing dataset:   2%|███                                                                                                                                                   | 1/50 [00:00<00:44,  1.11it/s]

Trying to call OpenAI API...


Processing dataset:   4%|██████                                                                                                                                                | 2/50 [00:01<00:38,  1.26it/s]

...

Processing dataset:  96%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████      | 48/50 [00:35<00:01,  1.48it/s]

Trying to call OpenAI API...


Processing dataset:  98%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████   | 49/50 [00:36<00:00,  1.46it/s]

Trying to call OpenAI API...


Processing dataset: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:37<00:00,  1.35it/s]


In [54]:
df_for_binary.head()

,pmid,journal_name,title,abstract,keywords,accepted_label,multi_label,binary_label,input_journal_title_abstract,input_title_abstract,gpt_predictions_P1_raw,gpt_predictions_P1,gpt_predictions_P2_raw,gpt_predictions_P2,gpt_predictions_P3_raw,gpt_predictions_P3
idx,,,,,,,,,,,,,,,,
530,24507711,Journal of clinical neuroscience : official jo...,Validation of the Chinese version of the NUCOG...,We aimed to develop and validate a Chinese ver...,Cognition| Dementia| Epilepsy| Mini-Mental Sta...,Human-RCT-non-intervention,13,0,<journal>Journal of clinical neuroscience : of...,<title>Validation of the Chinese version of th...,"{\n ""gpt_label"": ""OTHER""\n}",OTHER,"{\n ""gpt_label"": ""OTHER""\n}",OTHER,"{\n ""gpt_label"": ""OTHER""\n}",OTHER
404,28684367,World neurosurgery,Endovascular Therapy for the Steal Phenomenon ...,Symptomatic innominate artery stenosis present...,CAS| Carotid artery stenosis| Innominate arter...,Human-case-report,4,0,<journal>World neurosurgery</journal><title>En...,<title>Endovascular Therapy for the Steal Phen...,"{\n ""gpt_label"": ""OTHER""\n}",OTHER,"{\n ""gpt_label"": ""OTHER""\n}",OTHER,"{\n ""gpt_label"": ""OTHER""\n}",OTHER
465,2840539,Life sciences,Elevation of naloxone-sensitive 3H-dihydromorp...,3H-Dihydromorphine (DHM) binding sites were me...,NaN,Animal-other,5,1,<journal>Life sciences</journal><title>Elevati...,<title>Elevation of naloxone-sensitive 3H-dihy...,"{\n ""gpt_label"": ""OTHER""\n}",OTHER,"{\n ""gpt_label"": ""OTHER""\n}",OTHER,"{\n ""gpt_label"": ""OTHER""\n}",OTHER
143,35332187,Scientific reports,Effects of Apolipoprotein E polymorphism on ca...,The Apolipoprotein E (APOE) gene polymorphism ...,NaN,Remaining,0,0,<journal>Scientific reports</journal><title>Ef...,<title>Effects of Apolipoprotein E polymorphis...,"{\n ""gpt_label"": ""OTHER""\n}",OTHER,"{\n ""gpt_label"": ""OTHER""\n}",OTHER,"{\n ""gpt_label"": ""OTHER""\n}",OTHER
522,26582408,JMIR research protocols,Western Australian Public Opinions of a Minimu...,BACKGROUND: Excessive alcohol consumption has ...,addiction| alcohol| minimum pricing| policy,Clinical-study-protocol,12,0,<journal>JMIR research protocols</journal><tit...,<title>Western Australian Public Opinions of a...,"{\n ""gpt_label"": ""OTHER""\n}",OTHER,"{\n ""gpt_label"": ""OTHER""\n}",OTHER,"{\n ""gpt_label"": ""OTHER""\n}",OTHER


In [55]:
df_for_binary[["pmid","gpt_predictions_P2"]].groupby("gpt_predictions_P2").count()

,pmid
gpt_predictions_P2,
ANIMAL,6
OTHER,44


In [57]:
df_for_binary[["pmid","gpt_predictions_P3"]].groupby("gpt_predictions_P3").count()

,pmid
gpt_predictions_P3,
ANIMAL,8
OTHER,42


In [58]:
df_for_binary[["pmid","binary_label"]].groupby("binary_label").count()

,pmid
binary_label,
0,39
1,11
